In [1]:
import pandas as pd
import sys
import numpy as np
from sklearn.datasets import load_svmlight_file, dump_svmlight_file
sys.path.append('/home/ak/GitRepos/MKLpy')
sys.path.append('/home/ak/GitRepos/MKLpy/metrics/')

In [2]:
from MKLpy.regularization import normalization,rescale_01
from MKLpy.metrics.pairwise import HPK_kernel
from MKLpy.regularization import kernel_centering, kernel_normalization, tracenorm
from MKLpy.algorithms import EasyMKL,RMGD,RMKL,AverageMKL

from sklearn.metrics.pairwise import rbf_kernel as RBF

ImportError: No module named 'evaluate'